# 1. Импорт модулей.

In [ ]:
from google.colab import drive
from tensorflow.keras.layers import *
from tensorflow.keras.metrics import *
from tensorflow.keras.models import Sequential, Model
import numpy as np
import pandas as pd
import os
import tensorflow.keras.backend as K
from sklearn.metrics import *
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
from sklearn.preprocessing import MinMaxScaler
import pickle
import seaborn as sns
from matplotlib import pyplot as plt 
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report
from tensorflow.keras.models import load_model

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


#2. Чтение датасета и отброс Benign

In [ ]:
df_x = pd.read_csv('/content/drive/MyDrive/CICIDS2017/2022. Подключение к СИАСТ/x_smt.csv')
#df_x.rename(columns={" Label": "label"}, inplace = True)
df_x = df_x[df_x.label != "BENIGN"] 
print(df_x.head(1))
#df_x.to_csv('/content/drive/MyDrive/x_smt_no_benign', index=False)
#file = pd.read_csv('/content/drive/MyDrive/x_smt_no_benign')
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
target = le.fit_transform(df_x["label"])
data = df_x.drop("label", axis=1)
print(data.head(1))
le.get_params(deep = True)
print(le.classes_)

# 3. Решейпинг.


In [ ]:
data = np.array(data)
target = np.array(target)
print(data.shape, target.shape)

In [ ]:
#scaler = pickle.load(open('/content/drive/MyDrive/CICIDS2017/devtest/scaler.pkl', 'rb'))

In [ ]:
scaler = MinMaxScaler(feature_range=(-1,1))
scaler.fit(np.nan_to_num(data).astype(float))
data = scaler.transform(np.nan_to_num(data).astype(float))
print(data.shape)
data = data.reshape(data.shape[0], data.shape[1], 1)
print(data.shape)

# 4. Моделирование мультиклассовой нейросети KNN.


Моделирование мультиклассовой нейросети

In [ ]:
tf.keras.backend.clear_session()

knn = Sequential()
knn.add(InputLayer((data.shape[1],)))
knn.add(BatchNormalization())
for i in range(6):
  knn.add(Dense(128, activation='relu'))
knn.add(Dense(14, activation='softmax'))
#optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
knn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')
knn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 23)               92        
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 128)               3072      
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dense_3 (Dense)             (None, 128)               16512     
                                                                 
 dense_4 (Dense)             (None, 128)               16512     
                                                        

Тренировка второй нейросети

In [ ]:
knn.fit(data.astype(float), target, epochs=100, batch_size=256, verbose=1)
#pickle.dump(knn, open('/content/drive/MyDrive/CICIDS2017/2022. Подключение к СИАСТ/knn.pkl', 'wb'))
#knn.save('/content/drive/MyDrive/CICIDS2017/devtest/knn.h5')

Epoch 1/100
554/554 [==============================] - 6s 9ms/step - loss: 0.5823 - accuracy: 0.7648
Epoch 2/100
554/554 [==============================] - 5s 9ms/step - loss: 0.3962 - accuracy: 0.8189
Epoch 3/100
554/554 [==============================] - 5s 9ms/step - loss: 0.3609 - accuracy: 0.8317
Epoch 4/100
554/554 [==============================] - 5s 8ms/step - loss: 0.3560 - accuracy: 0.8316
Epoch 5/100
554/554 [==============================] - 5s 8ms/step - loss: 0.3455 - accuracy: 0.8352
Epoch 6/100
554/554 [==============================] - 5s 9ms/step - loss: 0.3382 - accuracy: 0.8368
Epoch 7/100
554/554 [==============================] - 5s 8ms/step - loss: 0.3375 - accuracy: 0.8370
Epoch 8/100
554/554 [==============================] - 5s 8ms/step - loss: 0.3412 - accuracy: 0.8372
Epoch 9/100
554/554 [==============================] - 5s 8ms/step - loss: 0.3306 - accuracy: 0.8404
Epoch 10/100
554/554 [==============================] - 5s 8ms/step - loss: 0.3310 - accura

In [ ]:
pickle.dump(knn, open('/content/drive/MyDrive/CICIDS2017/2022. Подключение к СИАСТ/knn.pkl', 'wb'))

Получение данных со второй нейросети

In [ ]:
y_pred1 = knn.predict(data)
y_pred = np.argmax(y_pred1, axis=1)

Вычисление метрик и количество классов

In [ ]:
target_names = le.classes_
#target_names = ['Bot', 'DDoS', 'DoS GoldenEye', 'DoS Hulk', 'DoS Slowhttptest', 'DoS slowloris', 'FTP-Patator', 'Heartbleed', 'Infiltration', 'PortScan', 'SSH-Patator', 'Web Attack � Brute Force', 'Web Attack � Sql Injection', 'Web Attack � XSS']
print('==============================Classification Report==============================')
print(classification_report(target, y_pred, target_names=target_names))
print('==============================Confussion Matrix and metrics==============================')
print('recall:', recall_score(target, y_pred, average='weighted'))
print('precision:', precision_score(target, y_pred, average='weighted'))
print('accuracy:', accuracy_score(target, y_pred))
print('f1 score:', f1_score(target, y_pred, average='weighted'))
print('ROC AUC:', roc_auc_score(to_categorical(target), np.nan_to_num(y_pred1), multi_class='ovr'))
print('Confussion Matrix:', "\n", confusion_matrix(target, y_pred))

==============================Classification Report==============================
                            precision    recall  f1-score   support

                       Bot       1.00      1.00      1.00     10241
                      DDoS       0.99      0.97      0.98     10222
             DoS GoldenEye       1.00      1.00      1.00     10242
                  DoS Hulk       0.94      0.71      0.81     10223
          DoS Slowhttptest       0.99      0.94      0.96     10228
             DoS slowloris       0.93      0.87      0.90     10225
               FTP-Patator       0.91      0.99      0.95     10239
                Heartbleed       1.00      1.00      1.00     10242
              Infiltration       1.00      0.99      0.99     10241
                  PortScan       1.00      1.00      1.00     10240
               SSH-Patator       0.56      0.99      0.71     10237
  Web Attack � Brute Force       0.82      0.15      0.25      9443
Web Attack � Sql Injection       